In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import json

In [98]:
like_post_emb = pd.read_csv("temp/post_embedding_topk_recent.csv", 
                            converters={"post_user": json.loads, "post_referrer": json.loads}, 
                            dtype={"postId": "str"})

like_referrer_emb = pd.read_csv("temp/referrer_embedding.csv", 
                            converters={"referrer_user": json.loads, "referrer_post": json.loads}, 
                            dtype={"postId": "str"})

In [99]:
like_referrer_emb.head()

,referrer,referrer_post,referrer_user,bias
0,trendingfeed,"[0.125759, 0.0495678, 0.167552, 0.239372, 0.21...","[0.135955, -0.118784, -0.262282, 0.127439, 0.5...",-0.603217
1,trendingfeed_suggested,"[0.376098, 0.131211, 0.241533, 0.179406, 0.019...","[0.159258, 0.146605, -0.120892, 0.0477811, 0.1...",-1.376080
2,videofeed,"[0.14437, 0.26674, 0.0474356, 0.113566, 0.1484...","[0.305248, 0.223542, -0.0928301, 0.425555, 0.0...",0.810465
3,videofeed_suggested,"[0.0413112, -0.00552638, 0.403039, 0.116875, 0...","[0.233801, 0.203753, 0.220149, 0.33223, 0.2173...",-0.474416


In [56]:
like_emb.dtypes

mapping            int64
postId            object
post_user         object
bias             float64
time              object
post_referrer     object
dtype: object

In [100]:
class FFM(tf.Module):
    def __init__(self, post_user_embs, post_referrer_embs, post_biases, postIds, 
                 referrer_post_embs, referrer_user_embs, referrer_biases, referrers, global_bias, K):
        self.post_user_embs = tf.constant(post_user_embs, dtype=tf.float32)
        self.post_referrer_embs = tf.constant(post_referrer_embs, dtype=tf.float32)
        self.post_biases = tf.constant(post_biases, dtype=tf.float32)
        self.postIds = tf.constant(postIds, dtype='string')
        self.global_bias = tf.constant(global_bias)
        self.numPosts = len(postIds)
        self.postIdToIndexTable = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(self.postIds, tf.constant(list(range(self.numPosts)), dtype='int64')),
            default_value=-1
        )
        self.referrer_user_embs = tf.constant(referrer_user_embs, dtype=tf.float32)
        self.referrer_post_embs = tf.constant(referrer_post_embs, dtype=tf.float32)
        self.referrer_biases = tf.constant(referrer_biases, dtype=tf.float32)
        self.referrers = referrers
        self.num_referrers = len(referrers)
        self.referrerToIndexTable = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(self.referrers, tf.constant(list(range(self.num_referrers)), dtype='int64')),
            default_value=-1
        )
        self.K = K

    @tf.function(input_signature=[
        tf.TensorSpec([32], tf.float32, name="user_post_emb"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb"),
        tf.TensorSpec([], tf.float32, name="user_bias"),
        tf.TensorSpec([], "string", name="referrer"),
        tf.TensorSpec([None], "string", name="seen_posts"),
        tf.TensorSpec([], "bool", name="normalize"),
    ])
    def predict(self, user_post_emb, user_referrer_emb, user_bias,  
                referrer, seen_posts, normalize):
        
        ind_ref = self.referrerToIndexTable.lookup(referrer)
        referrer_post_emb = tf.gather(self.referrer_post_embs, indices=ind_ref)
        referrer_user_emb = tf.gather(self.referrer_user_embs, indices=ind_ref)
        referrer_bias = tf.gather(self.referrer_biases, indices=ind_ref)
 
        if user_bias == -10000 or referrer_bias == -10000:
            return tf.zeros([1], dtype='float32'), tf.constant([], dtype="string")
               
        dot_user_post = tf.linalg.matmul(user_post_emb[tf.newaxis], self.post_user_embs, transpose_b=True)[0]
        dot_referrer_post = tf.linalg.matmul(referrer_post_emb[tf.newaxis], self.post_referrer_embs, transpose_b=True)[0]
        dot_user_referrer = tf.tensordot(user_referrer_emb, referrer_user_emb, 1)

        out = None
        if normalize:
            x = 0.7071067811865475
            out = 0.5 * (dot_user_post + dot_referrer_post + dot_user_referrer) + \
                x * self.post_biases + self.global_bias + x * user_bias + x * referrer_bias
        else:
            out = dot_user_post + dot_referrer_post + dot_user_referrer + \
                self.post_biases + self.global_bias + user_bias + referrer_bias

        seen_post_ind = self.postIdToIndexTable.lookup(seen_posts)
        seen_post_ind = seen_post_ind[seen_post_ind >= 0]

        mask = ~tf.sparse.to_dense(tf.sparse.reorder(
            tf.sparse.SparseTensor(seen_post_ind[:, tf.newaxis], tf.ones_like(seen_post_ind, dtype='bool'), [self.numPosts])
        ), default_value=False)
        masked_out = out[mask]
        masked_postIds = self.postIds[mask]

        k_to_get = self.K
        if tf.shape(masked_out)[0] < k_to_get:
            k_to_get = tf.shape(masked_out)[0]

        topk_ind = tf.math.top_k(masked_out, k = k_to_get).indices
        topk_post_ids = tf.gather(masked_postIds, topk_ind)

        return tf.sigmoid(out), topk_post_ids

In [103]:
def create_model(post_df, referrer_df):
    return FFM(post_df.post_user.values.tolist(), post_df.post_referrer.values.tolist(), post_df.bias.values.tolist(), post_df.postId.values, 
               referrer_df.referrer_post.values.tolist(), referrer_df.referrer_user.values.tolist(), referrer_df.bias.values.tolist(), referrer_df.referrer.values,
               0.1, 5500)

In [104]:
like_model = create_model(like_post_emb, like_referrer_emb)

In [105]:
user_post_emb = tf.constant([-0.116804, -0.314196, -0.628917, 0.0147506, 0.243619, 0.4848, -0.127038, 0.0912797,
                        -0.0973673, -0.320615, 0.187426, -0.00600598, -0.226693, 0.230844, 0.105143, 0.28843,
                        0.583322, 0.388846, -0.030855, -0.236351, -0.429838, 0.354583, 0.126397, 0.604772, -0.408072,
                        0.0176221, -0.106141, 0.34593, -0.159998, 0.291105, -0.237833, -0.306905])
user_bias = tf.constant(-0.21973)
user_referrer_emb = user_post_emb
referrer="trendingfeed"
seen_postIds = tf.constant(["1000007152", "9999579752", "7234678234", "1221213"])

In [106]:
t1, t2 = like_model.predict(user_post_emb, user_referrer_emb, user_bias, referrer,
    ["9400064252", "9999579752", "7234678234"],
    True
)

In [107]:
t1,t2

(<tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.4801759 , 0.26152575, 0.7752912 , ..., 0.7254201 , 0.74732995,
        0.72917265], dtype=float32)>,
 <tf.Tensor: shape=(3215,), dtype=string, numpy=
 array([b'9590029692', b'1472450792', b'1971802892', ..., b'7721649792',
        b'7668849692', b'5829675792'], dtype=object)>)

In [132]:
class CombinedFFMVideo(tf.Module):
    def __init__(self, like_model, share_model, fav_model, vplay_model, K):
        self.models = {
            "like": like_model,
            "share": share_model,
            "fav": fav_model,
            "vplay": vplay_model,
        }
        self.K = K

    @tf.function
    def getPostScores(self, postIds, model_out, model):
        indexes = model.postIdToIndexTable.lookup(postIds) + 1
        with_sentinel = tf.concat([tf.constant([0.0], dtype="float32"), model_out], 0)
        scores = tf.gather(with_sentinel, indexes)
        return scores

    @tf.function(input_signature=[
        tf.TensorSpec([None], "string", name="seen_posts"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_like"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_like"),
        tf.TensorSpec([], tf.float32, name="user_bias_like"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_share"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_share"),
        tf.TensorSpec([], tf.float32, name="user_bias_share"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_fav"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_fav"),
        tf.TensorSpec([], tf.float32, name="user_bias_fav"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_vplay"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_vplay"),
        tf.TensorSpec([], tf.float32, name="user_bias_vplay"),
        tf.TensorSpec([], "string", name="referrer"),        
        tf.TensorSpec([], tf.float32, name="m_like"),
        tf.TensorSpec([], tf.float32, name="c_like"),
        tf.TensorSpec([], tf.float32, name="m_share"),
        tf.TensorSpec([], tf.float32, name="c_share"),
        tf.TensorSpec([], tf.float32, name="m_fav"),
        tf.TensorSpec([], tf.float32, name="c_fav"),
        tf.TensorSpec([], tf.float32, name="m_vplay"),
        tf.TensorSpec([], tf.float32, name="c_vplay"),
        tf.TensorSpec([], "string", name="op"),
        tf.TensorSpec([], "bool", name="normalize"),
    ])
    def call(
        self, seen_posts,
        user_post_emb_like, user_referrer_emb_like, user_bias_like, 
        user_post_emb_share, user_referrer_emb_share, user_bias_share,
        user_post_emb_fav, user_referrer_emb_fav, user_bias_fav,
        user_post_emb_vplay, user_referrer_emb_vplay, user_bias_vplay,
        referrer,
        m_like, c_like,
        m_share, c_share,
        m_fav, c_fav,
        m_vplay, c_vplay,
        op, normalize,
    ):
        inputs = {
            "like" : { "user_post_emb": user_post_emb_like , "user_referrer_emb": user_referrer_emb_like,
                        "user_bias": user_bias_like, "m": m_like, "c": c_like },
            "share" : { "user_post_emb": user_post_emb_share , "user_referrer_emb": user_referrer_emb_share,
                        "user_bias": user_bias_share, "m": m_share, "c": c_share },
            "fav" : { "user_post_emb": user_post_emb_fav , "user_referrer_emb": user_referrer_emb_fav,
                        "user_bias": user_bias_fav, "m": m_fav, "c": c_fav },
            "vplay" : { "user_post_emb": user_post_emb_vplay , "user_referrer_emb": user_referrer_emb_vplay,
                        "user_bias": user_bias_vplay,"m": m_vplay, "c": c_vplay },
        }

        output_map = {}
        posts_to_get = tf.constant([[]], dtype="string")
        for event_name, inp in inputs.items():
            model_out, model_topk_posts = self.models[event_name].predict(inp["user_post_emb"], inp["user_referrer_emb"], inp["user_bias"],
                                                                          referrer, seen_posts, normalize)
            output_map[event_name] = model_out
            posts_to_get = tf.sets.union(posts_to_get, model_topk_posts[tf.newaxis])

        posts_to_get = tf.sparse.to_dense(posts_to_get)[0]

        scores_dic = {}
        transformed_scores = []
        for event_name, inp in inputs.items():
            scores = tf.zeros_like(posts_to_get, dtype="float32")
            if inp["user_bias"] != -10000:
                scores = self.getPostScores(posts_to_get, output_map[event_name], self.models[event_name])

            scores_dic[event_name] = scores
            transformed_scores.append(tf.pow(scores, inp["m"]) + inp["c"])


        combined_score = transformed_scores[0]
        if op == "mul":
            for i in range(1, len(transformed_scores)):
                combined_score *= transformed_scores[i]
        else:
            for i in range(1, len(transformed_scores)):
                combined_score += transformed_scores[i]

        k_to_get = self.K
        if tf.shape(combined_score)[0] < k_to_get:
            k_to_get = tf.shape(combined_score)[0]

        topk_obj = tf.math.top_k(combined_score, k = k_to_get)
        topk_ind = topk_obj.indices
        topk_combined_score = topk_obj.values

        to_ret = {
            "combined_scores": topk_combined_score,
            "postIds": tf.gather(posts_to_get, topk_ind)
        }

        for event_name in inputs:
            to_ret[f"{event_name}_scores"] = tf.gather(scores_dic[event_name], topk_ind)

        return to_ret

class CombinedFFMImage(tf.Module):
    def __init__(self, like_model, share_model, fav_model, lpo_model, K):
        self.models = {
            "like": like_model,
            "share": share_model,
            "fav": fav_model,
            "lpo": lpo_model,
        }
        self.K = K

    @tf.function
    def getPostScores(self, postIds, model_out, model):
        indexes = model.postIdToIndexTable.lookup(postIds) + 1
        with_sentinel = tf.concat([tf.constant([0.0], dtype="float32"), model_out], 0)
        scores = tf.gather(with_sentinel, indexes)
        return scores

    @tf.function(input_signature=[
        tf.TensorSpec([None], "string", name="seen_posts"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_like"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_like"),
        tf.TensorSpec([], tf.float32, name="user_bias_like"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_share"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_share"),
        tf.TensorSpec([], tf.float32, name="user_bias_share"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_fav"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_fav"),
        tf.TensorSpec([], tf.float32, name="user_bias_fav"),
        tf.TensorSpec([32], tf.float32, name="user_post_emb_lpo"),
        tf.TensorSpec([32], tf.float32, name="user_referrer_emb_lpo"),
        tf.TensorSpec([], tf.float32, name="user_bias_lpo"),
        tf.TensorSpec([], "string", name="referrer"),
        tf.TensorSpec([], tf.float32, name="m_like"),
        tf.TensorSpec([], tf.float32, name="c_like"),
        tf.TensorSpec([], tf.float32, name="m_share"),
        tf.TensorSpec([], tf.float32, name="c_share"),
        tf.TensorSpec([], tf.float32, name="m_fav"),
        tf.TensorSpec([], tf.float32, name="c_fav"),
        tf.TensorSpec([], tf.float32, name="m_lpo"),
        tf.TensorSpec([], tf.float32, name="c_lpo"),
        tf.TensorSpec([], "string", name="op"),
        tf.TensorSpec([], "bool", name="normalize"),
    ])
    def call(
        self, seen_posts,
        user_post_emb_like, user_referrer_emb_like, user_bias_like, 
        user_post_emb_share, user_referrer_emb_share, user_bias_share,
        user_post_emb_fav, user_referrer_emb_fav, user_bias_fav,
        user_post_emb_lpo, user_referrer_emb_lpo, user_bias_lpo,
        referrer,
        m_like, c_like,
        m_share, c_share,
        m_fav, c_fav,
        m_lpo, c_lpo,
        op, normalize,
    ):
        inputs = {
            "like" : { "user_post_emb": user_post_emb_like , "user_referrer_emb": user_referrer_emb_like,
                        "user_bias": user_bias_like, "m": m_like, "c": c_like },
            "share" : { "user_post_emb": user_post_emb_share , "user_referrer_emb": user_referrer_emb_share,
                        "user_bias": user_bias_share, "m": m_share, "c": c_share },
            "fav" : { "user_post_emb": user_post_emb_fav , "user_referrer_emb": user_referrer_emb_fav,
                        "user_bias": user_bias_fav, "m": m_fav, "c": c_fav },
            "lpo" : { "user_post_emb": user_post_emb_lpo , "user_referrer_emb": user_referrer_emb_lpo,
                        "user_bias": user_bias_lpo, "m": m_lpo, "c": c_lpo },
        }

        output_map = {}
        posts_to_get = tf.constant([[]], dtype="string")
        for event_name, inp in inputs.items():
            model_out, model_topk_posts = self.models[event_name].predict(inp["user_post_emb"], inp["user_referrer_emb"], inp["user_bias"],
                                                                          referrer, seen_posts, normalize)
            output_map[event_name] = model_out
            posts_to_get = tf.sets.union(posts_to_get, model_topk_posts[tf.newaxis])

        posts_to_get = tf.sparse.to_dense(posts_to_get)[0]

        scores_dic = {}
        transformed_scores = []
        for event_name, inp in inputs.items():
            scores = tf.zeros_like(posts_to_get, dtype="float32")
            if inp["user_bias"] != -10000:
                scores = self.getPostScores(posts_to_get, output_map[event_name], self.models[event_name])

            scores_dic[event_name] = scores
            transformed_scores.append(tf.pow(scores, inp["m"]) + inp["c"])


        combined_score = transformed_scores[0]
        if op == "mul":
            for i in range(1, len(transformed_scores)):
                combined_score *= transformed_scores[i]
        else:
            for i in range(1, len(transformed_scores)):
                combined_score += transformed_scores[i]

        k_to_get = self.K
        if tf.shape(combined_score)[0] < k_to_get:
            k_to_get = tf.shape(combined_score)[0]

        topk_obj = tf.math.top_k(combined_score, k = k_to_get)
        topk_ind = topk_obj.indices
        topk_combined_score = topk_obj.values

        to_ret = {
            "combined_scores": topk_combined_score,
            "postIds": tf.gather(posts_to_get, topk_ind)
        }

        for event_name in inputs:
            to_ret[f"{event_name}_scores"] = tf.gather(scores_dic[event_name], topk_ind)

        return to_ret


In [133]:
like_model = create_model(like_post_emb, like_referrer_emb)
share_model = create_model(like_post_emb, like_referrer_emb)
fav_model = create_model(like_post_emb, like_referrer_emb)
vplay_model = create_model(like_post_emb, like_referrer_emb)
# vclick_model = create_model(like_emb)
# vskip_model = create_model(vplay_emb)

In [134]:
model = CombinedFFMVideo(like_model, share_model, fav_model, vplay_model, 6000)

In [135]:
user_emb = [-0.116804, -0.314196, -0.628917, 0.0147506, 0.243619, 0.4848, -0.127038, 0.0912797,
    -0.0973673, -0.320615, 0.187426, -0.00600598, -0.226693, 0.230844, 0.105143, 0.28843,
    0.583322, 0.388846, -0.030855, -0.236351, -0.429838, 0.354583, 0.126397, 0.604772, -0.408072,
    0.0176221, -0.106141, 0.34593, -0.159998, 0.291105, -0.237833, -0.306905]
user_bias = -0.21973
m, c = 0.1, 0.1

In [136]:
# model.call(
#     ["9400064252", "9999579752", "7234678234"],
#     user_emb, user_bias, 
#     user_emb, user_bias, 
#     user_emb, user_bias, 
#     user_emb, user_bias,
#     m, c,
#     m, c,
#     m, c,
#     m, c,
#     "mul"
# )

In [137]:
model.call(
    ["9400064252", "9999579752", "7234678234"],
    user_emb, user_emb, user_bias,
    user_emb, user_emb, user_bias,
    user_emb, user_emb, user_bias,
    user_emb, user_emb, user_bias,
    "trendingfeed",
    m, c,
    m, c,
    m, c,
    m, c,
    "mul", True
)

{'combined_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([1.4053823 , 1.3971455 , 1.3922077 , ..., 0.76882887, 0.7576094 ,
        0.7121066 ], dtype=float32)>,
 'postIds': <tf.Tensor: shape=(3215,), dtype=string, numpy=
 array([b'9590029692', b'1472450792', b'1971802892', ..., b'7721649792',
        b'7668849692', b'5829675792'], dtype=object)>,
 'like_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.89349145, 0.8791487 , 0.87061995, ..., 0.16753116, 0.16077647,
        0.1351535 ], dtype=float32)>,
 'share_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.89349145, 0.8791487 , 0.87061995, ..., 0.16753116, 0.16077647,
        0.1351535 ], dtype=float32)>,
 'fav_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.89349145, 0.8791487 , 0.87061995, ..., 0.16753116, 0.16077647,
        0.1351535 ], dtype=float32)>,
 'vplay_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.89349145, 0.8791487 , 0.8706

In [138]:
like_model = create_model(like_post_emb, like_referrer_emb)
share_model = create_model(like_post_emb, like_referrer_emb)
fav_model = create_model(like_post_emb, like_referrer_emb)
lpo_model = create_model(like_post_emb, like_referrer_emb)
model = CombinedFFMImage(like_model, share_model, fav_model, lpo_model, 6000)
user_emb = [-0.116804, -0.314196, -0.628917, 0.0147506, 0.243619, 0.4848, -0.127038, 0.0912797,
    -0.0973673, -0.320615, 0.187426, -0.00600598, -0.226693, 0.230844, 0.105143, 0.28843,
    0.583322, 0.388846, -0.030855, -0.236351, -0.429838, 0.354583, 0.126397, 0.604772, -0.408072,
    0.0176221, -0.106141, 0.34593, -0.159998, 0.291105, -0.237833, -0.306905]
user_bias = -0.21973
m, c = 0.1, 0.1

In [141]:
model.call(
    ["9400064252", "9999579752", "7234678234"],
    user_emb, user_emb, user_bias,
    user_emb, user_emb, user_bias,
    user_emb, user_emb, user_bias,
    user_emb, user_emb, user_bias,
    "videofeed",
    m, c,
    m, c,
    m, c,
    m, c,
    "mul", True
)

{'combined_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([1.4443527 , 1.4413873 , 1.4394374 , ..., 1.000277  , 0.98020893,
        0.96024704], dtype=float32)>,
 'postIds': <tf.Tensor: shape=(3215,), dtype=string, numpy=
 array([b'1971802892', b'9590029692', b'9747729892', ..., b'5037761792',
        b'5838399792', b'5829675792'], dtype=object)>,
 'like_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.9633392 , 0.9579072 , 0.95434654, ..., 0.3489469 , 0.32984   ,
        0.3115021 ], dtype=float32)>,
 'share_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.9633392 , 0.9579072 , 0.95434654, ..., 0.3489469 , 0.32984   ,
        0.3115021 ], dtype=float32)>,
 'fav_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.9633392 , 0.9579072 , 0.95434654, ..., 0.3489469 , 0.32984   ,
        0.3115021 ], dtype=float32)>,
 'lpo_scores': <tf.Tensor: shape=(3215,), dtype=float32, numpy=
 array([0.9633392 , 0.9579072 , 0.954346

## Scrap code

In [76]:
module_path = 'out/tfserve_model'
tf.saved_model.save(model, module_path, signatures=model.call)

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None,), dtype=tf.string, name='postIds'), TensorSpec(shape=(None,), dtype=tf.float32, name='model_out'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fc99bb7e7d0>), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None,), dtype=tf.string, name='postIds'), TensorSpec(shape=(None,), dtype=tf.float32, name='model_out'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fc97e780610>), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None,), dtype=tf.string, name='postIds'), TensorSpec(shape=(None,), dtype=tf.float32, name='model_out'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fc96dbe8f50>), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None,), dtype=tf.string, name='postIds'), TensorSpec(shape=(None,), dtype=tf.float32, name='model_out'), <tens

In [62]:
K = 6000
models = {
    "like": like_model,
    "share": share_model,
    "fav": fav_model,
    "vplay": vplay_model,
}
seen_posts = ["9400064252", "9999579752", "7234678234"] 
op = "mul"
normalize = True
def getPostScores(postIds, model_out, model):
    indexes = model.postIdToIndexTable.lookup(postIds) + 1
    with_sentinel = tf.concat([tf.constant([0.0], dtype="float32"), model_out], 0) 
    scores = tf.gather(with_sentinel, indexes)
    return scores
    


inputs = {
    "like" : { "emb": user_emb , "bias": user_bias, "m": m, "c": c },
    "share": { "emb": user_emb, "bias": user_bias, "m": m, "c": c },
    "fav": { "emb": user_emb, "bias": user_bias, "m": m, "c": c },
    "vplay": { "emb": user_emb, "bias": user_bias, "m": m, "c": c},
}

output_map = {}
posts_to_get = tf.constant([[]], dtype="string")
for event_name, inp in inputs.items():
    model_out, model_topk_posts = models[event_name].predict(inp["emb"], inp["bias"], seen_posts, normalize)
    output_map[event_name] = model_out
    posts_to_get = tf.sets.union(posts_to_get, model_topk_posts[tf.newaxis])
    print(posts_to_get.shape)

posts_to_get = tf.sparse.to_dense(posts_to_get)[0]

scores_dic = {}
transformed_scores = []
for event_name, inp in inputs.items():
    scores = tf.zeros_like(posts_to_get, dtype="float32")
    if inp["bias"] != -10000:
        scores = getPostScores(posts_to_get, output_map[event_name], models[event_name])

    scores_dic[event_name] = scores
    transformed_scores.append(tf.pow(scores, inp["m"]) + inp["c"])


# combined_score = transformed_scores[0]
# if op == "mul":
#     for i in range(1, len(transformed_scores)):
#         combined_score *= transformed_scores[i]
# else:
#     for i in range(1, len(transformed_scores)):
#         combined_score += transformed_scores[i]

# k_to_get = self.K
# if tf.shape(combined_score)[0] < k_to_get:
#     k_to_get = tf.shape(combined_score)[0]

# topk_obj = tf.math.top_k(combined_score, k = k_to_get)
# topk_ind = topk_obj.indices
# topk_combined_score = topk_obj.values

# to_ret = {
#     "combined_scores": topk_combined_score,
#     "postIds": tf.gather(posts_to_get, topk_ind)
# }

# for event_name in inputs:
#     to_ret[f"{event_name}_scores"] = tf.gather(scores_dic[event_name], topk_ind)


(1, 2000)
(1, 2000)
(1, 2000)
(1, 4000)


In [64]:
transformed_scores

[<tf.Tensor: shape=(4000,), dtype=float32, numpy=array([0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1], dtype=float32)>,
 <tf.Tensor: shape=(4000,), dtype=float32, numpy=array([0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1], dtype=float32)>,
 <tf.Tensor: shape=(4000,), dtype=float32, numpy=array([0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1], dtype=float32)>,
 <tf.Tensor: shape=(4000,), dtype=float32, numpy=
 array([1.0385315, 1.0412363, 1.0437424, ..., 1.0153527, 1.0360458,
        1.0221673], dtype=float32)>]

In [52]:
posts_to_get = tf.sparse.to_dense(posts_to_get)

In [54]:
posts_to_get.

<tf.Tensor: shape=(1, 4000), dtype=string, numpy=
array([[b'1000415692', b'1003305692', b'1004236792', ..., b'9993484792',
        b'9993901792', b'9993983892']], dtype=object)>

In [65]:
scores

<tf.Tensor: shape=(4000,), dtype=float32, numpy=
array([0.53025967, 0.54574126, 0.5604473 , ..., 0.41293782, 0.5163816 ,
       0.44473073], dtype=float32)>

In [70]:
transformed_scores[3][:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1.0385315, 1.0412363, 1.0437424, 1.0143034, 1.0462399, 0.1      ,
       1.035229 , 1.0154598, 1.0567183, 1.014617 ], dtype=float32)>

In [71]:
postIds, model_out, model = posts_to_get, output_map[event_name], models[event_name]
indexes = model.postIdToIndexTable.lookup(postIds) + 1
with_sentinel = tf.concat([tf.constant([0.0], dtype="float32"), model_out], 0) 
scores = tf.gather(with_sentinel, indexes)

In [72]:
indexes

<tf.Tensor: shape=(4000,), dtype=int64, numpy=array([  90,   91, 4288, ..., 2814, 2003, 4915])>

In [73]:
model_out

<tf.Tensor: shape=(5408,), dtype=float32, numpy=
array([0.61646205, 0.22575873, 0.37112284, ..., 0.60106343, 0.5848813 ,
       0.6952676 ], dtype=float32)>

In [74]:
with_sentinel

<tf.Tensor: shape=(5409,), dtype=float32, numpy=
array([0.        , 0.61646205, 0.22575873, ..., 0.60106343, 0.5848813 ,
       0.6952676 ], dtype=float32)>

In [76]:
scores

<tf.Tensor: shape=(4000,), dtype=float32, numpy=
array([0.53025967, 0.54574126, 0.5604473 , ..., 0.41293782, 0.5163816 ,
       0.44473073], dtype=float32)>

In [77]:
with_sentinel[90]

<tf.Tensor: shape=(), dtype=float32, numpy=0.53025967>

In [79]:
 class CombinedFFMVideoWithViewClick(tf.Module):
    def __init__(self, like_model, share_model, fav_model, vplay_model, vclick_model, vskip_model, K):
        self.models = {
            "like": like_model,
            "share": share_model,
            "fav": fav_model,
            "vplay": vplay_model,
            "vclick": vclick_model,
            "vskip": vskip_model
        }
        self.K = K

    @tf.function
    def getPostScores(self, postIds, model_out, model):
        indexes = model.postIdToIndexTable.lookup(postIds) + 1
        with_sentinel = tf.concat([tf.constant([0.0], dtype="float32"), model_out], 0)
        scores = tf.gather(with_sentinel, indexes)
        return scores

    @tf.function(input_signature=[
        tf.TensorSpec([None], "string", name="seen_posts"),
        tf.TensorSpec([32], tf.float32, name="user_emb_like"),
        tf.TensorSpec([], tf.float32, name="user_bias_like"),
        tf.TensorSpec([32], tf.float32, name="user_emb_share"),
        tf.TensorSpec([], tf.float32, name="user_bias_share"),
        tf.TensorSpec([32], tf.float32, name="user_emb_fav"),
        tf.TensorSpec([], tf.float32, name="user_bias_fav"),
        tf.TensorSpec([32], tf.float32, name="user_emb_vplay"),
        tf.TensorSpec([], tf.float32, name="user_bias_vplay"),
        tf.TensorSpec([32], tf.float32, name="user_emb_vclick"),
        tf.TensorSpec([], tf.float32, name="user_bias_vclick"),
        tf.TensorSpec([32], tf.float32, name="user_emb_vskip"),
        tf.TensorSpec([], tf.float32, name="user_bias_vskip"),
        tf.TensorSpec([], tf.float32, name="m_like"),
        tf.TensorSpec([], tf.float32, name="c_like"),
        tf.TensorSpec([], tf.float32, name="m_share"),
        tf.TensorSpec([], tf.float32, name="c_share"),
        tf.TensorSpec([], tf.float32, name="m_fav"),
        tf.TensorSpec([], tf.float32, name="c_fav"),
        tf.TensorSpec([], tf.float32, name="m_vplay"),
        tf.TensorSpec([], tf.float32, name="c_vplay"),
        tf.TensorSpec([], tf.float32, name="m_vclick"),
        tf.TensorSpec([], tf.float32, name="c_vclick"),
        tf.TensorSpec([], tf.float32, name="m_vskip"),
        tf.TensorSpec([], tf.float32, name="c_vskip"),
        tf.TensorSpec([], "string", name="op"),
        tf.TensorSpec([], "bool", name="normalize"),
    ])
    def call(
        self, seen_posts,
        user_emb_like, user_bias_like,
        user_emb_share, user_bias_share,
        user_emb_fav, user_bias_fav,
        user_emb_vplay, user_bias_vplay,
        user_emb_vclick, user_bias_vclick,
        user_emb_vskip, user_bias_vskip,
        m_like, c_like,
        m_share, c_share,
        m_fav, c_fav,
        m_vplay, c_vplay,
        m_vclick, c_vclick,
        m_vskip, c_vskip,
        op, normalize,
    ):
        inputs = {
            "like" : { "emb": user_emb_like , "bias": user_bias_like, "m": m_like, "c": c_like },
            "share": { "emb": user_emb_share, "bias": user_bias_share, "m": m_share, "c": c_share },
            "fav": { "emb": user_emb_fav, "bias": user_bias_fav, "m": m_fav, "c": c_fav },
            "vplay": { "emb": user_emb_vplay, "bias": user_bias_vplay, "m": m_vplay, "c": c_vplay },
            "vclick": { "emb": user_emb_vclick, "bias": user_bias_vclick, "m": m_vclick, "c": c_vclick },
            "vskip": { "emb": user_emb_vskip, "bias": user_bias_vskip, "m": m_vskip, "c": c_vskip }
        }

        output_map = {}
        posts_to_get = tf.constant([[]], dtype="string")
        for event_name, inp in inputs.items():
            model_out, model_topk_posts = self.models[event_name].predict(inp["emb"], inp["bias"], seen_posts, normalize)
            output_map[event_name] = model_out
            posts_to_get = tf.sets.union(posts_to_get, model_topk_posts[tf.newaxis])

        posts_to_get = tf.sparse.to_dense(posts_to_get)[0]

        scores_dic = {}
        transformed_scores = []
        for event_name, inp in inputs.items():
            scores = tf.zeros_like(posts_to_get, dtype="float32")
            if inp["bias"] != -10000:
                scores = self.getPostScores(posts_to_get, output_map[event_name], self.models[event_name])

            scores_dic[event_name] = scores
            if(event_name != "vskip"):
                transformed_scores.append(tf.pow(scores, inp["m"]) + inp["c"])

        combined_score = transformed_scores[0]
        if op == "mul":
            for i in range(1, len(transformed_scores)):
                combined_score *= transformed_scores[i]
        else:
            for i in range(1, len(transformed_scores)):
                combined_score += transformed_scores[i]

        k_to_get = self.K
        if tf.shape(combined_score)[0] < k_to_get:
            k_to_get = tf.shape(combined_score)[0]

        topk_obj = tf.math.top_k(combined_score, k = k_to_get)
        topk_ind = topk_obj.indices
        topk_combined_score = topk_obj.values

        to_ret = {
            "combined_scores": topk_combined_score,
            "postIds": tf.gather(posts_to_get, topk_ind)
        }

        for event_name in inputs:
            to_ret[f"{event_name}_scores"] = tf.gather(scores_dic[event_name], topk_ind)

        return to_ret

In [81]:
model = CombinedFFMVideoWithViewClick(like_model, share_model, fav_model, vplay_model, 
                                      vclick_model, vskip_model, 6000)

In [30]:
res = model.call(
    ["9400064252", "9999579752", "7234678234"],
    user_emb, user_bias, 
    user_emb, user_bias, 
    user_emb, user_bias, 
    user_emb, user_bias,
    user_emb, user_bias,
    user_emb, user_bias,
    m, c,
    m, c,
    m, c,
    m, c,
    m, c,
    m, c,
    "mul", True
)

In [32]:
res

{'postIds': <tf.Tensor: shape=(200,), dtype=string, numpy=
 array([b'5733104203', b'9479392003', b'7601066003', b'7278167892',
        b'3404034003', b'3903012003', b'9768772103', b'3227438003',
        b'1049712203', b'1306943103', b'5849632203', b'3744640992',
        b'7701268103', b'7449099992', b'5860009992', b'5325225003',
        b'3251077103', b'9361679103', b'5726702203', b'7344720103',
        b'1333864003', b'3631718992', b'7982102003', b'1680754992',
        b'5826867792', b'7894233992', b'7209781992', b'5160658003',
        b'5952612103', b'3592866992', b'9338883992', b'7394494003',
        b'1107821992', b'9187954992', b'1109300203', b'3129068003',
        b'9600542203', b'7240540003', b'9083759892', b'9876159892',
        b'1445720692', b'7379287892', b'9862926103', b'9362232203',
        b'5225375003', b'9091212992', b'7927932103', b'1495978892',
        b'1164148103', b'3809351103', b'7083314992', b'5860787003',
        b'7928340203', b'7629835103', b'5275576003', b'96

In [29]:
model = tf.saved_model.load("/home/rohitrr/temp/ffm_data_incremental_new_video_6hr_2021_05_04T10_47_16Z_950365_odia_video_model3/")

In [ ]:
tensorflow_graph.call()

In [10]:
import pandas as pd
import random

In [19]:
df = pd.read_csv("./temp/post_embedding_topk_recent.csv")
copy_list = df["post_user"].values.copy()

In [20]:
random.shuffle(copy_list)
df["post_referrer"] = copy_list

In [13]:
df = df.rename(columns = {"embs": "post_user"})

In [21]:
df.head(5)

,mapping,postId,post_user,bias,time,post_referrer
0,2191444,1158615692,"[0.125759, 0.0495678, 0.167552, 0.239372, 0.21...",-0.603217,2021-04-28 02:06:22,"[0.135955, -0.118784, -0.262282, 0.127439, 0.5..."
1,2191464,1180635692,"[0.376098, 0.131211, 0.241533, 0.179406, 0.019...",-1.376080,2021-04-28 02:06:22,"[0.159258, 0.146605, -0.120892, 0.0477811, 0.1..."
2,2191498,1219205692,"[0.14437, 0.26674, 0.0474356, 0.113566, 0.1484...",0.810465,2021-04-28 02:06:22,"[0.305248, 0.223542, -0.0928301, 0.425555, 0.0..."
3,2191508,1228015692,"[0.0413112, -0.00552638, 0.403039, 0.116875, 0...",-0.474416,2021-04-28 02:06:22,"[0.233801, 0.203753, 0.220149, 0.33223, 0.2173..."
4,2191517,1242194692,"[0.359404, 0.284902, -0.0323388, 0.260955, 0.3...",-0.192275,2021-04-28 02:06:22,"[0.345623, 0.50464, 0.151791, 0.0110272, 0.258..."


In [22]:
df.to_csv("./temp/post_embedding_topk_recent.csv", index = False)

In [84]:
df = pd.DataFrame(columns = ["referrer", "referrer_post", "referrer_user", "bias"])

In [92]:
df.referrer = ["trendingfeed", "trendingfeed_suggested", "videofeed", "videofeed_suggested"]
df.referrer_post = like_emb.post_user.head(len(df.referrer.values))
df.referrer_user = like_emb.post_referrer.head(len(df.referrer.values))
df.bias = like_emb.bias.head(len(df.referrer.values))

In [97]:
df.to_csv("./temp/referrer_embedding.csv", index=False)